In [ ]:
import random
import json

actions = ['click', 'purchase']
users = [f'user_{i}' for i in range(1, 101)]  # 100 пользователей

data = []
for _ in range(1000):  # Генерируем 1000 действий
    action = random.choice(actions)
    user = random.choice(users)
    data.append({'user': user, 'action': action})

# Сохраняем данные в JSON файл для отправки в Kafka
with open('data.json', 'w') as f:
    json.dump(data, f)

In [ ]:
from kafka import KafkaProducer
import json

producer = KafkaProducer(bootstrap_servers='localhost:9092',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

with open('data.json') as f:
    data = json.load(f)
    for record in data:
        producer.send('example_topic', value=record)

producer.flush()

In [ ]:
from kafka import KafkaConsumer
from collections import Counter

consumer = KafkaConsumer('example_topic',
                         bootstrap_servers='localhost:9092',
                         auto_offset_reset='earliest',
                         enable_auto_commit=True,
                         group_id='my-group',
                         value_deserializer=lambda x: json.loads(x.decode('utf-8')))

user_actions = Counter()

for message in consumer:
    user_actions[message.value['user']] += 1

# Выводим пользователей с наибольшим количеством действий
most_active_users = user_actions.most_common(10)
print("Пользователи с наибольшим количеством действий:")
for user, count in most_active_users:
    print(f"{user}: {count} действий")
